In [1]:
# messy imports to be tidied up later
import os
import xlutils, xypath
import databaker
import databaker.constants
from databaker.constants import *
import databaker.databakersolo as ds
from databaker.jupybakeutils import savepreviewhtml, savepreviewhtmlBAGS, writetechnicalCSV, procrows, TechnicalCSV, yield_dimension_values, DUPgenerate_header_row
from databaker.jupybakeutils import HDim


In [2]:
"""
bake --preview ILCH.py ilchtablestemplatensa.xls growth
bake --preview ILCH.py ilchtablestemplatensa.xls level

bake --preview ILCH.py ilchtablestemplatesa.xls growth
bake --preview ILCH.py ilchtablestemplatesa.xls level
"""

# input-output
inputfile = "ilchtablestemplatesa.xls"
outputfile = "%s.csv" % os.path.splitext(inputfile)[0]
previewfile = "preview.html"


In [3]:
print("Loading %s which has size %d bytes" % (inputfile, os.path.getsize(inputfile)))
tableset = xypath.loader.table_set(inputfile, extension='xls')
tabs = list(xypath.loader.get_sheets(tableset, "*"))
print("Table names: %s" % ", ".join([tab.name  for tab in tabs]))


Loading ilchtablestemplatesa.xls which has size 407552 bytes
Table names: INTRODUCTION, DEFINITIONS, 1. Industry level SA, 2. Sector level SA, 3. Industry annual growth SA, 4. Sector annual growth SA, 5. Industry quarterly growth SA, 6. Sector quarterly growth SA


In [4]:
savepreviewhtmlBAGS(tabs[2], previewfile)

opening file preview.html
table '1. Industry level SA' written


In [5]:
newtabs = []
for tab in tabs:
    tab_title = tab.excel_ref("A1")
    if tab_title.filter(contains_string("growth")):    
        newtabs.append(tab)
tab = newtabs[0]
#[tab.name  for tab in newtabs]

In [11]:
datarowdim[0]

In [13]:
conversionsegments = [ ]
for tab in tabs:
    tab_title = tab.excel_ref("A1")
    if not tab_title.filter(contains_string("growth")):    # this is the word that was in PARAMS(1)
        continue
        
    anchor = tab.filter("Period").assert_one()

    datarows = anchor.fill(DOWN).is_not_blank()
    datacols = anchor.shift(DOWN).fill(RIGHT).is_not_blank()

    # !!! This is where we rewrite the values for this dimension
    datarowdim = HDim(datarows, TIME, DIRECTLY, LEFT) 
    for c in datarows:
        datarowdim.cellvalueoverride[c._cell] = c.value[:6]
    # (could define that lookup in constructor, or with a callback function, but result will be the same: 
    #  setting this cellvalueoverride dict to what it needs to be)
        
    obs = datarows.waffle(datacols).is_not_blank()
    savepreviewhtmlBAGS(obs, previewfile)

    dimensions = [ 
        datarowdim,   # the dimension in the list
        HDim(datacols.parent(), "Costs", DIRECTLY, ABOVE), 
        HDim(anchor.fill(RIGHT).parent(), "SIC", CLOSEST, LEFT)
    ]

    if tab_title.filter(contains_string("year on year")):
        dimensions.append(HDim("Annual", "Growth Period", None, None))
        dimensions.append(HDim("Percent", MEASURETYPE, None, None))
    elif tab_title.filter(contains_string("quarter on quarter")):
        dimensions.append(HDim("Quarterly", "Growth Period", None, None))
        dimensions.append(HDim("Percent", MEASURETYPE, None, None))
    elif tab_title.filter(contains_string("growth rates")):
        dimensions.append(HDim("Annual", "Growth Period", None, None))
        dimensions.append(HDim("Percent", MEASURETYPE, None, None))
    else:
        assert 0 #tab.dimension(MEASURETYPE,"Index")
    if tab_title.filter(contains_string("Non-Seasonally")):
        dimensions.append(HDim("Not seasonally adjusted", "SA / NSA", None, None))
    else:
        dimensions.append(HDim("Seasonally adjusted", "SA / NSA", None, None))

    conversionsegment = (tab, dimensions, obs)
    conversionsegments.append(conversionsegment)
    
savepreviewhtml(conversionsegments[-1], previewfile)


opening file preview.html
table '3. Industry annual growth SA' written
opening file preview.html
table '4. Sector annual growth SA' written
opening file preview.html
table '5. Industry quarterly growth SA' written
opening file preview.html
table '6. Sector quarterly growth SA' written
opening file preview.html
table '6. Sector quarterly growth SA' written
javascript calculated


In [16]:
len(conversionsegments)

4

In [17]:
savepreviewhtml(conversionsegments[-1], previewfile)   # <---- preview thing

opening file preview.html
table '6. Sector quarterly growth SA' written
javascript calculated


In [18]:
print("converting and writing %d conversion segments into %s" % (len(conversionsegments), outputfile))
convertedrows = [ ]
for conversionsegment in conversionsegments:
    print("conversion segment size %d" % len(conversionsegment[2]))
    rows = procrows(conversionsegment)
    convertedrows.append(rows)
    
# Should detect errors and throw up preview with cells marked
# (though actually the errors manifest further upstream, normally)
    
# here we can do further sorting and postprocessing and allocating to different files 


converting and writing 4 conversion segments into ilchtablestemplatesa.csv
conversion segment size 5952
conversion segment size 2232
conversion segment size 6240
conversion segment size 2340


In [22]:
set(convertedrows[0][i][-2]  for i in range(len(convertedrows[0])))

{'2001Q1',
 '2001Q2',
 '2001Q3',
 '2001Q4',
 '2002Q1',
 '2002Q2',
 '2002Q3',
 '2002Q4',
 '2003Q1',
 '2003Q2',
 '2003Q3',
 '2003Q4',
 '2004Q1',
 '2004Q2',
 '2004Q3',
 '2004Q4',
 '2005Q1',
 '2005Q2',
 '2005Q3',
 '2005Q4',
 '2006Q1',
 '2006Q2',
 '2006Q3',
 '2006Q4',
 '2007Q1',
 '2007Q2',
 '2007Q3',
 '2007Q4',
 '2008Q1',
 '2008Q2',
 '2008Q3',
 '2008Q4',
 '2009Q1',
 '2009Q2',
 '2009Q3',
 '2009Q4',
 '2010Q1',
 '2010Q2',
 '2010Q3',
 '2010Q4',
 '2011Q1',
 '2011Q2',
 '2011Q3',
 '2011Q4',
 '2012Q1',
 '2012Q2',
 '2012Q3',
 '2012Q4',
 '2013Q1',
 '2013Q2',
 '2013Q3',
 '2013Q4',
 '2014Q1',
 '2014Q2',
 '2014Q3',
 '2014Q4',
 '2015Q1',
 '2015Q2',
 '2015Q3',
 '2015Q4',
 '2016Q1',
 '2016Q2'}

In [23]:

writetechnicalCSV(outputfile, conversionsegments)



writing 4 conversion segments into ilchtablestemplatesa.csv
conversionwrite segment size 5952
conversionwrite segment size 2232
conversionwrite segment size 6240
conversionwrite segment size 2340


[{'Costs': 'Labour Costs per Hour',
  'Growth Period': 'Annual',
  'SA / NSA': 'Seasonally adjusted',
  -9: 16.9,
  -1: 'Quarter',
  -6: 'Percent',
  'SIC': 'ILCH_A\nAgriculture, Forestry and Fishing',
  -2: '2001Q1'},
 {'Costs': 'Labour Costs per Hour',
  'Growth Period': 'Annual',
  'SA / NSA': 'Seasonally adjusted',
  -9: 4.1,
  -1: 'Quarter',
  -6: 'Percent',
  'SIC': 'ILCH_B\nMining and Quarrying',
  -2: '2001Q2'},
 {'Costs': 'Labour Costs per Hour',
  'Growth Period': 'Annual',
  'SA / NSA': 'Seasonally adjusted',
  -9: 1.8,
  -1: 'Quarter',
  -6: 'Percent',
  'SIC': 'ILCH_C1\nManufacturing - Food Products, Beverages and Tobacco ',
  -2: '2001Q3'},
 {'Costs': 'Labour Costs per Hour',
  'Growth Period': 'Annual',
  'SA / NSA': 'Seasonally adjusted',
  -9: 2.6,
  -1: 'Quarter',
  -6: 'Percent',
  'SIC': 'ILCH_C2\nManufacturing - Textiles, Leather and Clothing',
  -2: '2001Q4'},
 {'Costs': 'Labour Costs per Hour',
  'Growth Period': 'Annual',
  'SA / NSA': 'Seasonally adjusted',
  -